In [ ]:
# Install required packages
!pip install -q opencv-python-headless requests

import cv2
import os
import requests
from datetime import timedelta
from urllib.parse import urlparse
import shutil
import zipfile
from google.colab import files

# Configuration
video_url = "https://ia601201.us.archive.org/0/items/CamelFilters/Camel%20filters.mp4"
frame_interval_seconds = 1
temp_video_name = "temp_video.mp4"
output_dir = "temp_frames"
zip_output = "extracted_frames.zip"

# Extract base video name without extension, replace spaces with underscores
video_name = os.path.splitext(os.path.basename(urlparse(video_url).path))[0]
video_name = video_name.replace(" ", "_")

# Step 1: Download video
def download_video(url, output_path):
    print(f"Downloading video from {url}...")
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise Exception(f"Failed to download video: {response.status_code}")
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Video downloaded successfully.")

# Step 2: Extract frames with video name and timestamp in filename
def extract_frames(video_path, output_folder, interval_sec):
    print("Extracting frames...")
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    if not fps or fps <= 0:
        raise ValueError("Could not determine video FPS.")

    frame_interval = int(fps * interval_sec)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            timestamp_sec = int(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000)
            timestamp_str = str(timedelta(seconds=timestamp_sec))
            # Clean timestamp string to be filename-safe
            timestamp_clean = timestamp_str.replace(":", "-")
            frame_filename = f"{video_name}__frame_{saved_count:04d}__{timestamp_clean}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Saved {saved_count} frames to {output_folder}.")

# Step 3: Zip frames
def zip_frames(folder_path, zip_path):
    print(f"Zipping frames into {zip_path}...")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files_in_dir in os.walk(folder_path):
            for file in files_in_dir:
                full_path = os.path.join(root, file)
                zipf.write(full_path, arcname=file)
    print("Zipping complete.")

# Step 4: Download zip
def download_zip(zip_path):
    print(f"Preparing {zip_path} for download...")
    files.download(zip_path)

# Full pipeline
def run_pipeline():
    download_video(video_url, temp_video_name)
    extract_frames(temp_video_name, output_dir, frame_interval_seconds)
    zip_frames(output_dir, zip_output)

    # Clean up
    os.remove(temp_video_name)
    shutil.rmtree(output_dir)

    download_zip(zip_output)

# Run it
run_pipeline()

